
### map csrAs in motility experiment to species in human microbiome  
Two methods:
1) via Taxonomy  
    -get taxonomic IDS from both the csrAs in the motility experiments and get taxonomic IDS from all of the species in the human gut microbiome
2) via sequence similarity
    - find all csrA orthologs in the Hs gut bacterial and archeal species and find if there are any exact matches or close matches
1) get motility csrAs
2) get taxonomy info from 

In [15]:
import os
from Bio import Phylo
import pandas as pd
from jw_utils import jw_draw_tree as jdt
from jw_utils import parse_fasta as pfa

### get metedata for human gut microbiome  
FTP downloads:  
http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v2.0.1/  


In [16]:
metadata_df =  pd.read_csv('../gut_microbiome/genomes-all_metadata.tsv', sep = '\t')
metadata_df = metadata_df.set_index('Genome')


### Get bacterial and archeal phylogenetic trees

In [17]:
#paths
tree_dir = '../Hs_gut_phylotrees/'
path_to_bac_tree = os.path.join(tree_dir, 'bac120_iqtree.nwk')
path_to_arch_tree = os.path.join(tree_dir, 'ar122_iqtree.nwk')
path_to_csra_tsv = '../Data/csrs.tsv'

In [18]:
#get phylogenetic tree for gut microbiome
if not os.path.exists(tree_dir):
    os.makedirs(tree_dir)

!curl http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v2.0.1/phylogenies/bac120_iqtree.nwk > $path_to_bac_tree
!curl http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v2.0.1/phylogenies/ar122_iqtree.nwk > $path_to_arch_tree

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: ftp.ebi.ac.uk
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: ftp.ebi.ac.uk


### Get dictionaries that contian best hits for both the forward and reverse blasts for both subtilis and aeruginosa csrA

In [19]:

def best_hit_to_dict(path_to_rblast_results, best_hit_col_name=None):
    """Return a dict with the accession of query genome and the best hit in the source genome

    These are the results of the reciprical blast. For each genome, there was a best hit in the initial
    blastp. This best hit was reciprical blasted against the genome containing the original query. If best
    hit in this reciprical blastp is the initial query, then these proteins are likely orthologs.
    
    return (df): cols = [accession, protein_id]
    """
    files = os.listdir(path_to_rblast_results)
    paths = [os.path.join(path_to_rblast_results, file) for file in files]
    d = {}
    for path, file  in zip(paths, files):
        accession = file.replace('_blastpOut.txt', '')
        with open(path, 'r') as f:
            for i,line in enumerate(f):
                if line.startswith('>'):
                    prot_id = line.split(' ')[0][1:]
                    d[accession]=prot_id
                    break
    df = pd.DataFrame.from_dict(d, orient= 'index')
    if not best_hit_col_name:
        best_hit_col_name = 'besthit_prot_id'
    df.columns = [best_hit_col_name]               
    return df



best_rHits_subt_df = best_hit_to_dict('../all_rblast_results_subtilis/reciprocal_blast_results/',best_hit_col_name='rbest_hit_subt' )
best_fHits_subt_df = best_hit_to_dict('../all_rblast_results_subtilis/forward_blast_results/', best_hit_col_name='fbest_hit_subt')
best_rHits_pseudo_df = best_hit_to_dict('../all_rblast_results_Psuedo/reciprocal_blast_results/', best_hit_col_name='rbest_hit_pseudo')
best_fHits_pseudo_df = best_hit_to_dict('../all_rblast_results_Psuedo/forward_blast_results/', best_hit_col_name='fbest_hit_pseudo')


### Get two dfs of hs gut associeated species accessoins  
1) containing those species with doubly-verified csrAs (csrA ortholog in both subtilis AND aeuruginosa reciprocal best hit blast)
2) second list containing all species who had ortholog in at least one (subtilis OR aeuruginosa eciprocal best hit blast)  

In [20]:
df_subt = pd.merge(best_rHits_subt_df,best_fHits_subt_df, left_index=True, right_index=True)
df_pseudo = pd.merge(best_rHits_pseudo_df,best_fHits_pseudo_df, left_index=True, right_index=True)
df = pd.merge(df_subt,df_pseudo, left_index=True, right_index=True)
bs_csrA_prot_ID = 'NP_391417.1'
pa_csrA_prot_ID = 'NP_249596.1'

df_filt_s = df.loc[df['rbest_hit_subt'] == bs_csrA_prot_ID,:]
df_filt_p = df.loc[df['rbest_hit_pseudo'] == pa_csrA_prot_ID,:]
df_double_verified = df_filt_s.loc[df_filt_s['rbest_hit_pseudo'] == pa_csrA_prot_ID,:]
df_single_verified = df.loc[set(list(df_filt_s.index) +  list(df_filt_p.index)),:]
df_double_verified

,rbest_hit_subt,fbest_hit_subt,rbest_hit_pseudo,fbest_hit_pseudo
MGYG000003619,NP_391417.1,MGYG000003619_01053,NP_249596.1,MGYG000003619_01053
MGYG000003060,NP_391417.1,MGYG000003060_00284,NP_249596.1,MGYG000003060_00284
MGYG000000626,NP_391417.1,MGYG000000626_00410,NP_249596.1,MGYG000000626_00410
MGYG000002462,NP_391417.1,MGYG000002462_03497,NP_249596.1,MGYG000002462_03497
MGYG000002532,NP_391417.1,MGYG000002532_03127,NP_249596.1,MGYG000002532_03127
...,...,...,...,...
MGYG000000864,NP_391417.1,MGYG000000864_02155,NP_249596.1,MGYG000000864_02155
MGYG000001491,NP_391417.1,MGYG000001491_01283,NP_249596.1,MGYG000001491_01283
MGYG000001312,NP_391417.1,MGYG000001312_01817,NP_249596.1,MGYG000001312_01817
MGYG000001486,NP_391417.1,MGYG000001486_01494,NP_249596.1,MGYG000001486_01494


In [21]:
df_single_verified

,rbest_hit_subt,fbest_hit_subt,rbest_hit_pseudo,fbest_hit_pseudo
MGYG000001571,NP_391417.1,MGYG000001571_04093,NP_249596.1,MGYG000001571_04093
MGYG000004825,NP_391417.1,MGYG000004825_00595,NP_251954.1,MGYG000004825_00661
MGYG000001275,NP_391417.1,MGYG000001275_00964,NP_251201.1,MGYG000001275_00045
MGYG000002416,NP_391417.1,MGYG000002416_00501,NP_252928.1,MGYG000002416_01080
MGYG000003661,NP_391417.1,MGYG000003661_01348,NP_249596.1,MGYG000003661_01348
...,...,...,...,...
MGYG000002404,NP_391417.1,MGYG000002404_03669,NP_249596.1,MGYG000002404_03669
MGYG000002344,NP_391417.1,MGYG000002344_00755,NP_249596.1,MGYG000002344_00755
MGYG000002357,NP_391417.1,MGYG000002357_03771,NP_249596.1,MGYG000002357_03771
MGYG000000620,NP_391417.1,MGYG000000620_00120,NP_253641.1,MGYG000000620_00564


### Get the protein sequences for the putative csrAs in the Hs gut microbiome species

In [22]:
def get_protein_dict(accessions, all_blast_results_dir):
    """Return dict with {prot_id:seq}, where seq is the forward blast best hit"""
    seq_dict = {}
    query_dir = os.path.join(all_blast_results_dir, 'forward_best_hits')
    for acc in accessions:
        seq_dict.update(pfa.get_seq_dict(os.path.join(query_dir, acc+'.faa')))
    return seq_dict
    
    
seq_dict_pseudo = get_protein_dict(accessions  = list(df_filt_p.index), all_blast_results_dir = '../all_rblast_results_Psuedo')
seq_dict_subtilis = get_protein_dict(accessions = list(df_filt_s.index), all_blast_results_dir = '../all_rblast_results_subtilis')    

In [23]:
def separate_lineage(df):
    lins = list(metadata_df['Lineage'])
    d = {}
    for acc in df.index:
        lin = df.loc[acc, 'Lineage']
        rank_l =lin.split(';')
        d[acc] = []
        for rank in rank_l:
            d[acc].append(rank[3:]) 
    lineage_df = pd.DataFrame.from_dict(d, orient='index')
    lineage_df.columns = ['Domain', 'Phylum', 'Class', 'Order', 'Fine', 'Gold', 'Species']
    return df.merge(lineage_df, left_index=True, right_index=True)
    

In [24]:
path_to_csra_tsv = '../Data/csrs.tsv'
mot_csrA_df = pd.read_csv(path_to_csra_tsv, sep='\t')
mot_csrA_seqs_d = pfa.get_seq_dict('../Protein_seqs/csrA_sequences.faa')

In [25]:
d = {}
d.update(seq_dict_subtilis)
d.update(seq_dict_pseudo)
matches = {prot_id:{} for prot_id in mot_csrA_seqs_d}
for prot_id, seq in mot_csrA_seqs_d.items():
    for gut_prot_id, gut_seq in d.items():
        if seq == gut_seq:
            matches[prot_id][gut_prot_id] = seq
matches_e = {}
for mot_csrA_id, dic, in matches.items():
    if len(dic)!=0:
        for gut_csrA_id, seq in dic.items():
            matches_e[gut_csrA_id] = seq
species_exact_matches =  [prot_id.split('_')[0] for prot_id in matches_e.keys()]
df = metadata_df.loc[species_exact_matches,:]
df = separate_lineage(df)[['Genome_type', 'Domain', 'Phylum', 'Class', 'Order', 'Fine', 'Gold', 'Species']]        
df

,Genome_type,Domain,Phylum,Class,Order,Fine,Gold,Species
MGYG000002415,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus sp001722135
MGYG000002488,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus penneri
MGYG000001624,MAG,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus cibarius
MGYG000002514,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus mirabilis
MGYG000004813,MAG,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus sp003144505
...,...,...,...,...,...,...,...,...
MGYG000002465,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Yersinia,Yersinia massiliensis
MGYG000002335,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Yersinia,Yersinia enterocolitica
MGYG000001351,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Edwardsiella,Edwardsiella tarda
MGYG000002467,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Yersinia,Yersinia frederiksenii_C


In [26]:
set(list(df['Phylum']))

{'Campylobacterota',
 'Desulfobacterota',
 'Firmicutes',
 'Firmicutes_A',
 'Proteobacteria'}

## Commented out code below should only be run once because of high computational burden

## Make one blast database from all Hs gut microbiome representative species  
1) Concatenate proteomes from each species into one large file to make the database  

In [27]:
# proteomes = os.listdir('../Proteomes/')   
# for proteome in proteomes:   
#     fp = os.path.join('../Proteomes', proteome)    
#     !cat $fp >> '../giant_proteome.faa'  
#     !makeblastdb -in ../all_hs_gut_proteomes.faa -dbtype prot -out hs_gut_microbiome_db

In [28]:
# #write each mot csrA sequence to its own individual fasta file
# p_dir =  '../ind_csrA_fastas'
# os.makedirs(p_dir)
# for seq_id, seq in mot_csrA_seqs_d.items():
#     pfa.write_to_fasta({seq_id:seq}, os.path.join(p_dir, seq_id+'.faa'))

### Blastp each csrA query against the blastp database hs_gut_microbiome_db

In [34]:
def multiquery_blastp(individ_fasta_dir, out_p_dir,):   
    
    proteins = os.listdir(individ_fasta_dir)
    os.makedirs(p_dir)
    for protein in proteins:
        fp = os.path.join('../ind_csrA_fastas/', protein)
        out_filepath = os.path.join(out_p_dir, protein)
        base_filepath = '../hs_gut_microbiome_db/hs_gut_microbiome_db'
        print(fp, base_filepath, out_filepath)
        !blastp -query $fp -db $base_filepath -num_descriptions 50 -num_alignments 50 -word_size 7 -out $out_filepath
        print(f'finished blasting {protein} ')
        
        
out_p_dir = '../All_csrA_hits'
individ_fasta_dir = '../ind_csrA_fastas/'
#multiquery_blastp(individ_fasta_dir, out_p_dir,)

In [30]:
def get_perc_id_blast(fp, cutoff = 99):
    """Return the top hit and the percent identity to the query"""
    with open(fp, 'r') as f: 
        a = f.readlines()
    d = {}
    
    for i, ele in enumerate(a):
        if ele.startswith('>'):
            prot_id = a[i].split(' ')[0][1:]
            perc_ident = float(a[i+4].split(',')[0].split(' ')[-1].replace('(','').replace(')','')[:-1])
            if perc_ident>=cutoff:
                d[prot_id] = perc_ident

    return d


per_id_d = {}
p_dir = '../All_csrA_hits/'
files = os.listdir(p_dir)
for file in files:
    fp = os.path.join(p_dir, file)
    per_id_d[file.replace('.faa', '')] = get_perc_id_blast(fp, cutoff = 100)
df_blast_species = pd.DataFrame.from_dict(per_id_d, orient='index').transpose()
blast_species = set([ele.split('_')[0] for ele in list(df_blast_species.index)])

In [31]:
blast_species_df = separate_lineage(metadata_df.loc[blast_species,:])[['Genome_type', 'Domain', 'Phylum', 'Class', 'Order', 'Fine', 'Gold', 'Species']]
blast_species_df

,Genome_type,Domain,Phylum,Class,Order,Fine,Gold,Species
MGYG000002458,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Yersinia,Yersinia aleksiciae
MGYG000002519,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Enterobacter,Enterobacter asburiae_B
MGYG000002335,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Yersinia,Yersinia enterocolitica
MGYG000004813,MAG,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus sp003144505
MGYG000002507,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia albertii
...,...,...,...,...,...,...,...,...
MGYG000002520,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Pluralibacter,Pluralibacter gergoviae
MGYG000001624,MAG,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus cibarius
MGYG000002816,MAG,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Citrobacter,Citrobacter braakii
MGYG000000110,Isolate,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Kluyvera,Kluyvera sp902363335


The dominant gut microbial phyla are   
Firmicutes, Bacteroidetes, Actinobacteria, Proteobacteria, Fusobacteria, and Verrucomicrobia,  
with the two phyla Firmicutes and Bacteroidetes [13] representing 90% of gut microbiota.  
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6351938/

In [33]:
set(blast_species_df['Species'])

{'',
 'Acetivibrio_A ethanolgignens',
 'Agathobacter rectalis',
 'Agathobacter sp000434275',
 'Agathobacter sp900546625',
 'Agathobacter sp900550545',
 'Agathobaculum sp900291975',
 'Anaerotruncus colihominis',
 'Bacillus licheniformis',
 'Bacillus paralicheniformis',
 'CAG-303 sp000437755',
 'CAG-411 sp000437275',
 'Campylobacter_D jejuni',
 'Citrobacter braakii',
 'Citrobacter europaeus',
 'Citrobacter freundii',
 'Citrobacter gillenii',
 'Citrobacter portucalensis',
 'Citrobacter portucalensis_A',
 'Citrobacter youngae',
 'Citrobacter_A amalonaticus',
 'Citrobacter_A farmeri',
 'Citrobacter_A sedlakii',
 'Citrobacter_B koseri',
 'Cronobacter malonaticus',
 'Cronobacter sakazakii',
 'Edwardsiella tarda',
 'Eisenbergiella tayi',
 'Enterobacter asburiae_B',
 'Enterobacter cancerogenus',
 'Enterobacter chengduensis',
 'Enterobacter cloacae',
 'Enterobacter cloacae_K',
 'Enterobacter hormaechei_A',
 'Enterobacter kobei',
 'Enterobacter ludwigii',
 'Enterobacter mori',
 'Enterobacter rogg